In [2]:
import torch
from transformers import AutoModelForSequenceClassification
import pandas as pd
import numpy as np
import evaluate

ModuleNotFoundError: No module named 'evaluate'

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("VCNC/Auto-CNC")